## Setup

In [1]:
# Install spacy
!pip install -U pip setuptools wheel
!pip install -U spacy
!pip install transformers

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
Requirement already up-to-date: pip in /Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages (20.3.4)
     |████████████████████████████████| 583 kB 4.7 MB/s eta 0:00:01
Requirement already up-to-date: wheel in /Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages (0.37.1)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 41.2.0
    Uninstalling setuptools-41.2.0:
      Successfully uninstalled setuptools-41.2.0
DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no

In [4]:
import spacy
import pandas as pd
from collections import Counter
from spacy import displacy
from transformers import pipeline
!python -m spacy download en_core_web_md
nlp = spacy.load("en_core_web_sm")

Keyring is skipped due to an exception: 'keyring.backends'
  Using cached transformers-4.28.1-py3-none-any.whl (7.0 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 4.2 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 4.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.4/294.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 20.1
    Uninstalling packaging-20.1:
      Successfully uninstalled packaging-20.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.

[notice] A new release of pip available: 22.2.2

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure
    from your online store in Germany. Unfortunately, when I opened the package,
    I discovered to my horror that I had been sent an action figure of Megatron
    instead! As a lifelong enemy of the Decepticons, I hope you can understand my
    dilemma. To resolve the issue, I demand an exchange of Megatron for the
    Optimus Prime figure I ordered. Enclosed are copies of my records concerning
    this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

In [1]:
sentiment_analysis = pipeline("sentiment-analysis")
classifier = pipeline("text-classification")
ner_tagger = pipeline("ner", aggregation_strategy="simple")
summarizer = pipeline("summarization", clean_up_tokenization_spaces=True) #, max_length=45)

reader = pipeline("question-answering")
question = "What does the customer want?"
outputs1 = reader(question=question, context=text)
pd.DataFrame([outputs1])

NameError: name 'pipeline' is not defined

In [81]:
outputs2 = classifier(text)
pd.DataFrame(outputs2)

Token indices sequence length is longer than the specified maximum sequence length for this model (383159 > 512). Running this sequence through the model will result in indexing errors


KeyboardInterrupt: 

In [79]:
from pprint import pprint
pprint(text[500:1000])

('Credit Suisse funds in the week surrounding the shotgun wedding.\n'
 '\n'
 'The latest figures show an additional $2.5 billion of net outflows from '
 'Credit Suisse funds between March 23 and April 6.\n'
 '\n'
 '“It is fair to say that there is still uncertainty around the deal and the '
 'potential impact that it may have on Credit Suisse Asset Management,” Johann '
 'Scholtz, an equity analyst at Morningstar told FN.\n'
 '\n'
 'Heavy outflows from Credit Suisse funds began on March 14, the day the bank '
 'said it had discovered “materia')


In [19]:
cs_bodies = pd.read_csv('../data/bodies/bodies_Credit Suisse_Apr17.csv')
cs_bodies['body_type'] = cs_bodies['Body'].apply(lambda x: type(x))
cs_bodies = cs_bodies[cs_bodies['body_type'] == str]
cs_bodies['body_length_chr'] = cs_bodies['Body'].apply(lambda x: len(x))
body_lst = cs_bodies['Body']
text = ' '.join(cs_bodies['Body'])

In [87]:
test = 'av av dfv dfv safd sadf sd asdv asdv asdv asdv'

def analyze_sentiment(text):
    result = sentiment_analysis(text)[0]
    return result['label'], result['score']

sentiment, score = analyze_sentiment(test)

print(sentiment, score)


NEGATIVE 0.9840806126594543


In [42]:
def extract_top5_orgs(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    orgs = [ent for ent in entities if ent[1] == 'ORG']
    top5_orgs = list(pd.Series([org[0] for org in orgs]).value_counts().head(5).index)
    return top5_orgs



In [72]:
def summarizer(text):
    result = summarizer(text)[0]
    return result['label']

RecursionError: maximum recursion depth exceeded

In [60]:
print(f"5 most mentioned entities: {top5_orgs}")
print(f"Sentiment: {sentiment}, Score: {score}")

5 most mentioned entities: ['Credit Suisse', 'UBS', 'Reuters', 'SEC', "Credit Suisse's"]
Sentiment: NEGATIVE, Score: 0.9954714775085449


In [14]:
doc = nlp(cs_bodies['Body'][45])
sentence1 = list(doc.sents)[0]

In [21]:
# dependency graph
# displacy.render(sentence1, style="dep")

## Similarity

In [35]:
doc2 = nlp(cs_bodies['Body'][15])
doc2[:50]

Inflation may be weakening, but it’s not going away. That’s why pricing expert Adam Echter—who has spent almost every hour of the last three years first explaining to companies how a coming inflation would impact their business, and then helping them do something

In [36]:
doc[:50]

Simon-Kucher looks back at another spectacular year of growth, booking 21% growth in 2022 to help the global consultancy crack the half a billion euro revenue barrier.

Founded in 1985 by four high-profile German academics, Simon-Kucher today is an

In [37]:
doc.similarity(doc2)

0.832031002776474

## Entity recognition

In [ ]:
orgs = []

for ent in doc.ents:
    if ent.label_ == 'ORG':
        orgs.append(ent.text)
counted_list = Counter(orgs)

for item, count in counted_list.items():
    print(f"{item}: {count}")

# Visual representation of entities
#displacy.render(doc, style="ent")

### <font color='red'> **How to ensure entities are recongnized correctly using maybe lemmatization** </font>

## Pipelines

In [8]:
nlp = spacy.blank('en')

In [9]:
nlp.add_pipe('sentencizer')

In [15]:
doc_p = nlp(cs_bodies['Body'][45])

In [18]:
len(list(doc.sents))

19

### <font color='red'> **GPT2** </font>

In [ ]:
# Import necessary libraries
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer

# Load the dataset
dataset = load_dataset("text", data_files={"train": "path/to/your/news_data.txt"})

# Tokenize the dataset
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=1024)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Load the GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Define training arguments
training_args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_steps=500,
    save_steps=500,
    logging_steps=100,
    logging_dir="logs",
    evaluation_strategy="steps",
)

# Create Trainer instance and fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
)

trainer.train()

# Save the fine-tuned model
trainer.save_model("fine_tuned_gpt2")

# Load the fine-tuned model for text generation
tokenizer = GPT2Tokenizer.from_pretrained("fine_tuned_gpt2")
model = GPT2LMHeadModel.from_pretrained("fine_tuned_gpt2")

# Generate text with the fine-tuned model
input_text = "Some text about a news article or entity"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

output = model.generate(input_ids)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)
